In [1]:
## Imports

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import (RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, 
                              AdaBoostClassifier, BaggingClassifier, StackingClassifier)
import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss, classification_report, roc_auc_score, ConfusionMatrixDisplay


from sklearn.impute import SimpleImputer

sklearn.set_config(display='diagram')
%matplotlib inline

# Load Data and Train-Test-Split

In [4]:
# Load Meta Data
meta_data = pd.read_csv('data/train.csv')

# Load Data
signs = pd.read_csv('data/all_targets.csv',index_col=0)['0'].to_list()
signs_5 = ['hungry','stay','drink','dad','sad']
dataset_5_signs = pd.read_csv('data/dataset_hungry_stay_drink_dad_sad.csv', index_col=0)
dataset = pd.concat((dataset_5_signs,
                     pd.read_csv('data/dataset_round2_155.csv',index_col=0)), 
                    axis=0)
# Load sign name to int mapping
with open('data/sign_to_prediction_index_map.json') as f:
    sign_map = json.load(f)


# Train Test Split 5 signs
X = dataset_5_signs.drop(columns='sign')
y = dataset_5_signs['sign'].map({sign:i for i, sign in enumerate(signs_5)})
X_train_5_signs, X_test_5_signs, y_train_5_signs, y_test_5_signs = train_test_split(X, y, test_size=.25, stratify=y, random_state=42)
print(X_train_5_signs.shape, X_test_5_signs.shape, y_train_5_signs.shape, y_test_5_signs.shape)

# Train Test Split 87 signs
X = dataset.drop(columns='sign')
y = dataset['sign'].map({sign:i for i, sign in enumerate(signs)})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, stratify=y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(1447, 2172) (483, 2172) (1447,) (483,)


((45390, 2172), (15130, 2172), (45390,), (15130,))

# Load Model